In [20]:
import pandas as pd
import numpy as np
from pathlib import Path

import plotly.express as px
pd.options.plotting.backend = 'plotly'

import plotly.figure_factory as ff 

import seaborn as sns

In [21]:
#read dataset into df
abortions_raw = pd.read_excel('abortions.xlsx')
abortions_raw.head()

,U.S. State,"% change in abortion rate, 2017-2020","% change in the no. of abortion clinics, 2017-2020","% change in the no. of abortion providers, 2014-2017","% of all U.S. abortions, by state of occurrence, 2020","% of counties without a known abortion provider, 2014","% of counties without a known clinic, 2020","% of residents obtaining abortions who traveled out of state for care, 2020","% of women aged 15-44 living in a county without a clinic, 2020","% of women aged 15-44 living in a county without an abortion provider, 2014",...,"No. of abortions per 1,000 women aged 15–44, by state of occurrence, 2020","No. of abortions per 1,000 women aged 15–44, by state of residence, 2020","No. of abortions, by state of occurrence, 2020","No. of abortions, by state of residence, 2020","No. of federally funded abortions, 2010","No. of state funded abortions, 2010","Reported public expenditures for abortions (in 000s of dollars), federal, 2015","Reported public expenditures for abortions (in 000s of dollars), state, 2015","Total no. of publicly funded abortions , 2010","Total reported public expenditures for abortions (in 000s of dollars), 2015"
0,Alabama,-6,0,-22,0.6,91,93,47,59,58,...,6.0,9.5,5700,9060,9,0,22,0,9,22
1,Alaska,0,0,-25,0.1,79,87,7,33,17,...,8.6,9.2,1240,1320,0,835,0,216,835,216
2,Arizona,1,0,-8,1.4,80,80,6,18,19,...,9.3,9.7,13320,13820,1,13,28,11,14,40
3,Arkansas,2,-33,0,0.3,97,99,37,86,77,...,5.6,7.8,3250,4510,0,0,0,0,0,0
4,California,17,7,-18,16.6,24,38,0,3,1,...,19.2,19.0,154060,152400,0,88466,0,32613,88466,32613


In [22]:
# Quick manual mapping
state_to_region = {
    'Maine': 'Northeast',
    'Vermont': 'Northeast',
    'New Hampshire': 'Northeast',
    'Massachusetts': 'Northeast',
    'Connecticut': 'Northeast',
    'Rhode Island': 'Northeast',
    'New York': 'Northeast',
    'New Jersey': 'Northeast',
    'Pennsylvania': 'Northeast',

    'Ohio': 'Midwest',
    'Michigan': 'Midwest',
    'Indiana': 'Midwest',
    'Illinois': 'Midwest',
    'Wisconsin': 'Midwest',
    'Minnesota': 'Midwest',
    'Iowa': 'Midwest',
    'Missouri': 'Midwest',
    'North Dakota': 'Midwest',
    'South Dakota': 'Midwest',
    'Nebraska': 'Midwest',
    'Kansas': 'Midwest',

    'Delaware': 'South',
    'Maryland': 'South',
    'Virginia': 'South',
    'West Virginia': 'South',
    'Kentucky': 'South',
    'Tennessee': 'South',
    'North Carolina': 'South',
    'South Carolina': 'South',
    'Georgia': 'South',
    'Florida': 'South',
    'Alabama': 'South',
    'Mississippi': 'South',
    'Arkansas': 'South',
    'Louisiana': 'South',
    'Texas': 'South',
    'Oklahoma': 'South',

    'Montana': 'West',
    'Idaho': 'West',
    'Wyoming': 'West',
    'Colorado': 'West',
    'New Mexico': 'West',
    'Arizona': 'West',
    'Utah': 'West',
    'Nevada': 'West',
    'California': 'West',
    'Oregon': 'West',
    'Washington': 'West',
    'Alaska': 'West',
    'Hawaii': 'West'
}


In [23]:
abortions_raw.columns

Index(['U.S. State', '% change in abortion rate, 2017-2020',
       '% change in the no. of abortion clinics, 2017-2020',
       '% change in the no. of abortion providers, 2014-2017',
       '% of all U.S. abortions, by state of occurrence, 2020',
       '% of counties without a known abortion provider, 2014',
       '% of counties without a known clinic, 2020',
       '% of residents obtaining abortions who traveled out of state for care, 2020',
       '% of women aged 15-44 living in a county without a clinic, 2020',
       '% of women aged 15-44 living in a county without an abortion provider, 2014',
       'Abortion rate (the no. of abortions per 1,000 women aged 15-17), by state of residence, 2017',
       'Abortion rate (the no. of abortions per 1,000 women aged 15-19), by state of residence, 2017',
       'Abortion rate (the no. of abortions per 1,000 women aged 18-19), by state of residence, 2017',
       'Change in the no. of abortion clinics, 2017-2020',
       'Change in th

In [24]:
# For access declines
declines_raw = abortions_raw[['U.S. State', '% of counties without a known clinic, 2020', '% of counties without a known abortion provider, 2014']]

# For no. of abortions
abortions_2020 = abortions_raw[['U.S. State', 'No. of abortions, by state of occurrence, 2020']]


In [25]:
#declines = declines.rename(columns={'% of counties without a known clinic, 2020': '2020', '% of counties without a known abortion provider, 2014' : '2014'})
declines_raw.head()

,U.S. State,"% of counties without a known clinic, 2020","% of counties without a known abortion provider, 2014"
0,Alabama,93,91
1,Alaska,87,79
2,Arizona,80,80
3,Arkansas,99,97
4,California,38,24


In [26]:
#isolate top ten states with greatest increase in % of counties without clinic access
declines_raw['Change'] = declines_raw['% of counties without a known clinic, 2020'] - declines_raw['% of counties without a known abortion provider, 2014']


top_declines = declines_raw.sort_values(by='Change', ascending=False).iloc[:10, :]
top_declines = top_declines.drop(columns=['Change'])
top_declines

,U.S. State,"% of counties without a known clinic, 2020","% of counties without a known abortion provider, 2014"
21,Massachusetts,50,21
29,New Hampshire,60,40
11,Hawaii,40,20
30,New Jersey,29,14
4,California,38,24
45,Vermont,64,50
9,Florida,75,67
33,North Carolina,91,83
1,Alaska,87,79
20,Maryland,62,54


In [27]:
#clean for easier graphing
top_declines = pd.melt(top_declines, id_vars = 'U.S. State', var_name= 'Year', value_name= 'Percent of Counties without Clinic')
top_declines['Year'] = top_declines['Year'].replace({
    '% of counties without a known clinic, 2020' : '2020',
    '% of counties without a known abortion provider, 2014' : '2014'
})
top_declines['Percent of Counties with Clinic'] = 100 - top_declines['Percent of Counties without Clinic']
top_declines

,U.S. State,Year,Percent of Counties without Clinic,Percent of Counties with Clinic
0,Massachusetts,2020,50,50
1,New Hampshire,2020,60,40
2,Hawaii,2020,40,60
3,New Jersey,2020,29,71
4,California,2020,38,62
5,Vermont,2020,64,36
6,Florida,2020,75,25
7,North Carolina,2020,91,9
8,Alaska,2020,87,13
9,Maryland,2020,62,38


In [28]:
top_declines['Region'] = top_declines['U.S. State'].map(state_to_region)

In [29]:
top_declines['Region']

0     Northeast
1     Northeast
2          West
3     Northeast
4          West
5     Northeast
6         South
7         South
8          West
9         South
10    Northeast
11    Northeast
12         West
13    Northeast
14         West
15    Northeast
16        South
17        South
18         West
19        South
Name: Region, dtype: object

In [ ]:
opp_fig = px.histogram(
    top_declines, 
    x='U.S. State', 
    y='Percent of Counties with Clinic', 
    color='Year', 
    barmode='group', 
    category_orders={'Year': ['2014', '2020']},
    color_discrete_map={
        '2014': '#3871ab',
        '2020': '#ab3838'
    }
)

opp_fig.update_layout(
    title={
        'text': "Top 10 States with Largest Decrease in Counties With Abortion Clinics<br><sup>Data shown for top 10 states with largest decreases in the percentage of counties with known abortion clinics between 2014 and 2020; other states omitted for clarity.</sup>",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
    },
    yaxis_title='% of Counties with Clinic Access'
)

opp_fig.show()


In [54]:
opp_fig.write_html("/Users/cherylxiang/Desktop/dsc106/project2/propfinal.html")

In [32]:
#isolate top ten states with least increase in % of counties without clinic access

bot_declines = declines_raw.sort_values(by=['Change', '% of counties without a known clinic, 2020'], ascending=[True, False]).iloc[:10, :]
bot_declines = bot_declines.drop(columns=['Change'])
bot_declines

,U.S. State,"% of counties without a known clinic, 2020","% of counties without a known abortion provider, 2014"
19,Maine,12,75
47,Washington,56,94
6,Connecticut,0,13
26,Montana,91,93
13,Illinois,89,91
22,Michigan,86,88
24,Mississippi,99,99
25,Missouri,99,99
34,North Dakota,98,98
41,South Dakota,98,98


In [33]:
#clean for easier graphing
bot_declines = pd.melt(bot_declines, id_vars = 'U.S. State', var_name= 'Year', value_name= 'Percent of Counties without Clinic')
bot_declines['Year'] = bot_declines['Year'].replace({
    '% of counties without a known clinic, 2020' : '2020',
    '% of counties without a known abortion provider, 2014' : '2014'
})
bot_declines['Percent of Counties with Clinic'] = 100 - bot_declines['Percent of Counties without Clinic']
bot_declines
bot_declines

,U.S. State,Year,Percent of Counties without Clinic,Percent of Counties with Clinic
0,Maine,2020,12,88
1,Washington,2020,56,44
2,Connecticut,2020,0,100
3,Montana,2020,91,9
4,Illinois,2020,89,11
5,Michigan,2020,86,14
6,Mississippi,2020,99,1
7,Missouri,2020,99,1
8,North Dakota,2020,98,2
9,South Dakota,2020,98,2


In [34]:
prop_fig = px.histogram(bot_declines, 
                        x='U.S. State', 
                        y='Percent of Counties with Clinic', 
                        color='Year', 
                        barmode='group', 
                        category_orders={'Year': ['2014', '2020']},
                        color_discrete_map={
                            '2014': '#ab3838',
                            '2020': '#3871ab'
                        }
                        )

prop_fig.update_layout(
    title={
        'text': "Some US States Experienced Increases in Abortion Clinic Availibility<br><sup>Data shown for top 10 states with largest increases in the percentage of counties with known abortion clinics between 2014 and 2020; other states omitted for clarity.</sup>",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    yaxis_title='Percent of Counties with Clinic Access'
)

prop_fig.show()

In [35]:
abortions_2020.head()

,U.S. State,"No. of abortions, by state of occurrence, 2020"
0,Alabama,5700
1,Alaska,1240
2,Arizona,13320
3,Arkansas,3250
4,California,154060


In [36]:
abortions = abortions_raw.copy()

# Clean the data
abortions['U.S. State'] = abortions['U.S. State'].str.strip()
abortions['% change in the no. of abortion clinics, 2017-2020'] = pd.to_numeric(
    abortions['% change in the no. of abortion clinics, 2017-2020'], errors='coerce'
)
abortions_clean = abortions.dropna(subset=['U.S. State', '% change in the no. of abortion clinics, 2017-2020'])

# Sort by clinic change
abortions_sorted = abortions_clean.sort_values('% change in the no. of abortion clinics, 2017-2020')

# Create bar chart
fig = px.bar(
    abortions_sorted,
    x='U.S. State',
    y='% change in the no. of abortion clinics, 2017-2020',
    title="Percentage Change in Abortion Clinics by State (2017–2020)",
    labels={'% change in the no. of abortion clinics, 2017-2020': '% Change'},
    color='% change in the no. of abortion clinics, 2017-2020',
    color_continuous_scale='Reds'
)

fig.update_layout(xaxis_tickangle=-45)
fig.show()


In [37]:
# Sort by % change
abortions_sorted = abortions_clean.sort_values('% change in the no. of abortion clinics, 2017-2020')

# Select top 5 with biggest losses and bottom 5 with smallest losses or gains
top_10 = abortions_sorted.head(10)      # Biggest clinic % decreases
bottom_10 = abortions_sorted.tail(10)   # Smallest decreases or even increases

# Combine for plotting
extremes = pd.concat([top_10, bottom_10])


In [38]:
abortions_sorted['Region'] = abortions_sorted['U.S. State'].map(state_to_region)

In [39]:
increases = bottom_10.sort_values(by='% change in the no. of abortion clinics, 2017-2020', ascending=False)
increases['Region'] = increases['U.S. State'].map(state_to_region)

In [40]:
increases['State (Region)'] = increases.apply(lambda row: f"{row['U.S. State']} ({row['Region']})", axis=1)

In [41]:
increases['State (Region)']

17       Kentucky (South)
23    Minnesota (Midwest)
49    Wisconsin (Midwest)
28          Nevada (West)
5         Colorado (West)
36       Oklahoma (South)
13     Illinois (Midwest)
26         Montana (West)
14      Indiana (Midwest)
43          Texas (South)
Name: State (Region), dtype: object

In [43]:
fig5 = px.bar(
    increases,
    x='State (Region)',
    y='% change in the no. of abortion clinics, 2017-2020',
    color='% change in the no. of abortion clinics, 2017-2020',
    color_continuous_scale='Blues',
    title= "Gains in Abortion Clinic Access Across Regions (2017–2020)",
    labels={'% change in the no. of abortion clinics, 2017-2020': '% Change'}
)

fig5.update_layout(
    title={
        'text': "Gains in Abortion Clinic Access Across Regions (2017–2020)<br><sup>States from South, Midwest, and West led in percentage increases. Northeast not shown due to smaller relative changes.</sup>",
        'y': 0.95,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    xaxis_title='State',
    yaxis_title='% Change in Clinics',
    xaxis_tickangle=-45
)

fig5.show()

In [44]:
fig5.write_html("/Users/cherylxiang/Desktop/dsc106/project2/oppfinal.html")

In [45]:
fig = px.bar(
    extremes,
    x='U.S. State',
    y='% change in the no. of abortion clinics, 2017-2020',
    color='% change in the no. of abortion clinics, 2017-2020',
    color_continuous_scale='RdBu',
    title="States with Largest and Smallest Changes in Abortion Clinics (2017–2020)",
    labels={'% change in the no. of abortion clinics, 2017-2020': '% Change'}
)

fig.update_layout(
    xaxis_title='State',
    yaxis_title='% Change in Clinics',
    xaxis_tickangle=-45
)

fig.show()

In [46]:
filtered_extremes = abortions_clean[abortions_clean['% change in the no. of abortion clinics, 2017-2020'] > -70]

fig2 = px.histogram(
    filtered_extremes,
    x='% change in the no. of abortion clinics, 2017-2020',
    nbins=10,
    histnorm='percent',
    title="Most States Saw Modest Changes in Abortion Clinics",
    labels={'% change in the no. of abortion clinics, 2017-2020': '% Change in Clinics'},
    color_discrete_sequence=['#3871ab']
)

fig2.update_layout(
    title={
        'text': "Most States Saw Modest Changes in Abortion Clinics<br><sup>The majority experienced relatively small percentage changes from 2017 to 2020.</sup>",
        'y': 0.95,
        'x': 0.5,
        'xanchor': 'center'
    },
    xaxis_title='% Change in Clinics (2017–2020)',
    yaxis_title='Number of States',
    bargap=0.1
)

fig2.show()

In [47]:
changes_raw = abortions_raw[['U.S. State', '% of counties without a known clinic, 2020', '% of counties without a known abortion provider, 2014']]
changes_raw['Percent of Counties With Clinics 2020'] = 100 - changes_raw['% of counties without a known clinic, 2020']
changes_raw['Percent of Counties With Clinics 2014'] = 100 - changes_raw['% of counties without a known abortion provider, 2014']
changes_raw = changes_raw.drop(columns= ['% of counties without a known clinic, 2020', '% of counties without a known abortion provider, 2014'])
changes_raw['Percent Change'] = changes_raw['Percent of Counties With Clinics 2020'] - changes_raw['Percent of Counties With Clinics 2014']
changes_sorted = changes_raw.sort_values(by='Percent Change', ascending=True)
changes_sorted.head()

,U.S. State,Percent of Counties With Clinics 2020,Percent of Counties With Clinics 2014,Percent Change
21,Massachusetts,50,79,-29
29,New Hampshire,40,60,-20
11,Hawaii,60,80,-20
30,New Jersey,71,86,-15
4,California,62,76,-14


In [48]:

top_ten = changes_sorted.head(10)      # Biggest clinic % decreases
bottom_ten = changes_sorted.tail(10)   # Smallest decreases or even increases

# Combine for plotting
extreme_10s = pd.concat([top_ten, bottom_ten])

In [49]:
fig = px.bar(
    extreme_10s,
    x='U.S. State',
    y='Percent Change',
    color='Percent Change',
    color_continuous_scale='RdBu',
    title="States with Largest and Smallest Changes in Abortion Clinics (2014–2020)",
    labels = {'Percent Change': '% Change'}
)

fig.update_layout(
    xaxis_title='State',
    yaxis_title='% Change in Clinics',
    xaxis_tickangle=-45
)

fig.show()

In [50]:
fig3 = px.histogram(
    changes_sorted,
    x='Percent Change',
    nbins=10,
    title="Most States Saw Modest Changes in Abortion Clinics",
    labels={'% change in the no. of abortion clinics, 2014-2020': '% Change in Clinics'},
    color_discrete_sequence=['#3871ab']
)

fig3.update_layout(
    title={
        'text': "Most States Saw Modest Changes in Abortion Clinics<br><sup>The majority experienced relatively small percentage changes from 2017 to 2020.</sup>",
        'y': 0.95,
        'x': 0.5,
        'xanchor': 'center'
    },
    xaxis_title='% Change in Clinics (2014–2020)',
    yaxis_title='Number of States',
    bargap=0.1
)

fig3.show()

In [51]:
filtered_extremes = changes_sorted[changes_sorted['Percent Change'] > -70]

# Then plot proportions
fig4 = px.histogram(
    filtered_extremes,
    x='Percent Change',
    nbins=8,
    histnorm='percent',  # This changes it from counts to proportion!
    color_discrete_sequence=['#3871ab'],
    title="Distribution of Changes in Abortion Clinics Among Majority of U.S. States (2014–2020)"
)

fig4.update_layout(
    xaxis_title='% Change in Clinics',
    yaxis_title='Proportion of States (%)',
    bargap=0.1
)

fig4.show()
